In [ ]:
# --- Consistent BS vs MC pipeline (call option) -------------------------------
import yfinance as yf
import numpy as np
from datetime import datetime, timezone, timedelta
from math import log, sqrt, exp
from scipy.stats import norm
from scipy.optimize import brentq
import pandas as pd

# -------------------------
# Black–Scholes Call Price
# -------------------------
def bs_call_price(S, K, T, r, q, sigma):
    if T <= 0:
        raise ValueError(f"T must be > 0, got {T}")
    if sigma <= 0:
        raise ValueError(f"sigma must be > 0, got {sigma}")
    d1 = (log(S / K) + (r - q + 0.5 * sigma**2) * T) / (sigma * sqrt(T))
    d2 = d1 - sigma * sqrt(T)
    return S * exp(-q * T) * norm.cdf(d1) - K * exp(-r * T) * norm.cdf(d2)

# -------------------------
# Implied Volatility Solver
# -------------------------
def implied_vol_call(S, K, T, r, q, market_price, lo=1e-6, hi=5.0):
    def f(sig):
        return bs_call_price(S, K, T, r, q, sig) - market_price
    f_lo, f_hi = f(lo), f(hi)
    if f_lo * f_hi > 0:
        raise RuntimeError("IV solver failed to bracket the root.")
    return brentq(f, lo, hi, maxiter=200, xtol=1e-10)

# -------------------------
# Risk-neutral Monte Carlo (GBM)
# -------------------------
def mc_call_price(S0, K, T, r, q, sigma, paths=200_000, steps=252, seed=42):
    if T <= 0:
        raise ValueError("T must be > 0 for simulation.")
    rng = np.random.default_rng(seed)
    dt = T / steps
    mu = r - q
    Z = rng.standard_normal((paths, steps))
    increments = (mu - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z
    ST = S0 * np.exp(np.cumsum(increments, axis=1)[:, -1])
    payoff = np.maximum(ST - K, 0.0)
    disc = np.exp(-r * T)
    price = disc * payoff.mean()
    stderr = disc * payoff.std(ddof=1) / np.sqrt(paths)
    return price, stderr

# -------------------------
# Auto dividend yield (q)
# -------------------------
def fetch_dividend_yield_from_yf(tkr, S0=None):
    """
    Dividend yield q (decimal):
      1) fast_info['dividend_yield']  (ideal)
      2) TTM cash dividends / S0
      3) Last 4 dividends / S0
      4) Else 0.0
    """
    tk = yf.Ticker(tkr)

    # --- 1) fast_info
    try:
        fi = tk.fast_info
        dy = fi.get("dividend_yield", None)
        if dy is not None and np.isfinite(dy) and dy >= 0:
            return float(dy)
    except:
        pass

    # --- Ensure S0 exists
    if S0 is None or not np.isfinite(S0):
        try:
            S0 = float(tk.history(period="5d")["Close"].dropna().iloc[-1])
        except:
            return 0.0

    # --- 2) TTM cash dividends
    try:
        divs = tk.dividends
        if divs is not None and not divs.empty:
            divs.index = pd.to_datetime(divs.index).tz_localize(None)
            cutoff = pd.Timestamp.utcnow().tz_localize(None) - pd.Timedelta(days=365)
            ttm = float(divs[divs.index >= cutoff].sum())
            if ttm > 0:
                return ttm / S0

            # --- 3) Last 4 payouts (typical quarterly dividends)
            last4 = float(divs.tail(4).sum())
            if last4 > 0:
                return last4 / S0
    except:
        pass

    # No dividends
    return 0.0

# -------------------------
# Collecting S0, K, T, market price
# -------------------------
def fetch_inputs_from_yf(tkr, exp=None, use_put_chain=False):
    tk = yf.Ticker(tkr)

    S0 = float(tk.history(period="1d")["Close"].iloc[-1])

    all_exps = tk.options
    if not all_exps:
        raise RuntimeError("No listed options.")
    if exp is None:
        exp = all_exps[0]

    ch = tk.option_chain(exp)
    table = ch.puts if use_put_chain else ch.calls
    table = table.dropna(subset=["strike"])

    atm_idx = (table["strike"] - S0).abs().idxmin()
    row = table.loc[atm_idx]
    K = float(row["strike"])

    bid = float(row.get("bid", np.nan))
    ask = float(row.get("ask", np.nan))
    last = float(row.get("lastPrice", np.nan))

    if np.isfinite(bid) and np.isfinite(ask) and bid > 0 and ask > 0:
        market_price = 0.5 * (bid + ask)
    elif np.isfinite(last) and last > 0:
        market_price = last
    else:
        mark = float(row.get("mark", np.nan))
        if np.isfinite(mark) and mark > 0:
            market_price = mark
        else:
            raise RuntimeError("Could not determine market option price.")

    now = datetime.now(timezone.utc)
    exp_dt = datetime.fromisoformat(exp).replace(tzinfo=timezone.utc)
    T = (exp_dt - now).total_seconds() / (365.25 * 24 * 3600)
    if T <= 0:
        raise ValueError(f"T <= 0 for {exp}")

    return {
        "tkr": tkr,
        "S0": S0,
        "K": K,
        "exp": exp,
        "T": T,
        "market_price": market_price,
        "row": row,
        "chain_table": table
    }

# -------------------------
# Example run
# -------------------------
tkr = "AMZN"
r = 0.042  # 4.2% risk-free rate

data = fetch_inputs_from_yf(tkr)
S0 = data["S0"]; K = data["K"]; T = data["T"]; market_price = data["market_price"]

# AUTO dividend yield
q = fetch_dividend_yield_from_yf(tkr, S0=S0)

# IV from market price
iv = implied_vol_call(S0, K, T, r, q, market_price)

# Black–Scholes price
bs_price = bs_call_price(S0, K, T, r, q, iv)

# Monte Carlo price
mc_price, mc_se = mc_call_price(S0, K, T, r, q, iv)

print("Dividend yield q =", q)
print(f"Ticker: {tkr}")
print(f"S0={S0:.4f}, K={K:.4f}, T={T:.6f}, r={r:.4%}, q={q:.4%}")
print(f"Market option price ≈ {market_price:.4f}")
print(f"Implied Vol (call): {iv:.4%}")
print(f"BS price:          {bs_price:.4f}")
print(f"MC price:          {mc_price:.4f} (± {1.96*mc_se:.4f})")

Dividend yield q = 0.0
Ticker: AMZN
S0=248.8000, K=250.0000, T=0.008768, r=4.2000%, q=0.0000%
Market option price ≈ 2.7550
Implied Vol (call): 35.2057%
BS price:          2.7550
MC price:          2.7694 (± 0.0199)
